# Parkinson's Disease detection with saved trained models as an ensemble.

# 1. Import the dependencies.

In [ ]:
import numpy as np
import pandas as pd
from typing import Tuple
import random
import math
import librosa
import os
import pickle
from tensorflow.keras.models import load_model

# 2. Load the audios with Librosa, and preprocess them using Peak Amplitude Normalization.
- load_audio will return all the audios with their sampling rate.
- to_positive will turn all values to postive numbers (audio is 1D).
- get_max will return the maximum value in each audio array.
- peak_amplitude_normalize will preprocess audios with this formula 10 ** (peak / 20) / maximum_value.
- hamming_window uses this formula 0.54 - 0.46 * math.cos((2 * math.pi * n) / frame_length - 1) to every frame of the audio and returns windowed audios.
- frame_audio_signal uses hamming_window on frames of audios and returns framed audios.

In [ ]:
def load_audio(file_path: str) -> Tuple:
    audio, sampling_rate = librosa.load(file_path, sr=44100)
    return audio, sampling_rate

def to_positive(n_array):  # Turn all values to positive values
    for i in range(len(n_array)):
        if n_array[i] < 0:
            n_array[i] = -1 * n_array[i]
    return n_array

def get_max(n_array):  # Get the maximum value
    max_value = n_array[0]
    for i in range(1, len(n_array), 1):
        if n_array[i] > max_value:
            max_value = n_array[i]
    return max_value

def peak_amplitude_normalize(audio_data, peak=-3.0):  # Calculate a scaling factor based on the specific peak value
    n_array = to_positive(audio_data)               # (-3 dB) and multiply the entire audio signal by the scaling factor
    maximum_value = get_max(n_array)
    scaling = 10 ** (peak / 20) / maximum_value
    normalized_audio = audio_data * scaling
    return normalized_audio

def hamming_window(frame_length):
    window = np.zeros(frame_length)
    n_negative_one = frame_length - 1
    for n in range(frame_length):
        window[n] = 0.54 - 0.46 * math.cos((2 * math.pi * n) / n_negative_one)
    return window

# Function to frame the signal using a Hamming window
def frame_audio_signal(audio_data, frame_length):
    length_audio = len(audio_data)
    number_frames = length_audio // frame_length
    framed_audio = np.zeros((frame_length, number_frames))
    for i in range(number_frames):
        start = i * frame_length
        stop = start + frame_length
        samples = audio_data[start:stop] * hamming_window(frame_length)
        framed_audio[:, i] = samples
    return framed_audio

# 3. Extract audio features for saved models.
- audio_features_status extracts zero cross rating, energy, MFCC, spectral centroid and spectral rolloff for Logistic Regression, KNN, SVM, Decision Tree, and Random Forest respectively.

In [ ]:
def audio_features_status() -> Tuple:
    directory = '/content/drive/MyDrive/HYP TEST DATA/'
    zcr_features = []
    energy_features = []
    mfcc_features = []
    spectral_features = []
    audios = []
    status = []
    for file_name in os.listdir(directory):
        if file_name.endswith('.wav'):
            if "P" in file_name:
                status.append(1)
            if "C" in file_name:
                status.append(0)

            file_path = os.path.join(directory, file_name)
            audio, sampling_rate = load_audio(file_path)
            frame_time = 0.02  # Duration of each frame in seconds
            frame_size = int(frame_time * sampling_rate)  # Number of samples in each frame
            frame_length = int(5 * sampling_rate)
            preprocessed_audio = peak_amplitude_normalize(audio)
            frames = frame_audio_signal(preprocessed_audio, frame_size)

            zcr = np.mean(np.abs(np.diff(np.sign(frames))), axis=1)
            zcr_features.append(zcr.mean())

            framed_audio = frame_audio_signal(preprocessed_audio, frame_length)
            energy = np.sum(framed_audio ** 2, axis=0)
            energy_features.append(energy)

            mfcc = np.mean(librosa.feature.mfcc(y=preprocessed_audio, sr=sampling_rate, n_mfcc=13).T, axis=0)
            mfcc_features.append(mfcc)

            spectral_centroid = librosa.feature.spectral_centroid(y=preprocessed_audio, sr=sampling_rate).mean()
            spectral_roll_off = librosa.feature.spectral_rolloff(y=preprocessed_audio, sr=sampling_rate).mean()
            feats = np.array([spectral_centroid, spectral_roll_off])
            spectral_features.append(feats)

            audios.append(np.expand_dims(preprocessed_audio, axis=0))

    zcr_features, energy_features, mfcc_features, spectral_features, audios, status = np.array(zcr_features), np.array(energy_features), np.array(mfcc_features), np.array(spectral_features), np.array(audios), np.array(status)
    return zcr_features, energy_features, mfcc_features, spectral_features, audios, status

#### voting below takes majority voting of the predictions made by each of the loaded model.

In [ ]:
def voting(predicted_values):
     positive, negative = 0, 0
     for i in range(len(predicted_values)):
         if predicted_values[i] == 1:
             positive += 1
         else:
              negative += 1
     if positive >= negative:
         return 1
     else:
          return 0

# 4. Metrics
- _confusion_matrix computes true positives, fales positives, true negatives, and false negatives.
- accuracy_score, precision_score, recall_score, f1_score andd confusion_matrix compute accuracy, precison, recall, and f1 scores and confusion matrix.

In [ ]:
def _confusion_matrix(y_testing, y_prediction):
    # Computing confusion matrix
    length_of_labels = len(y_testing)
    true_positive, false_positive, true_negative, false_negative = 0, 0, 0, 0

    for i in range(length_of_labels):
        if y_testing[i] == 1:
            if y_testing[i] == y_prediction[i]:
                true_positive += 1

            else:
                false_positive += 1

        if y_testing[i] == 0:
            if y_testing[i] == y_prediction[i]:
                true_negative += 1

            else:
                false_negative += 1

    return true_positive, false_positive, true_negative, false_negative


def accuracy_score(y_testing, y_preds):
    tp, fp, tn, fn = _confusion_matrix(y_testing, y_preds)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    return accuracy


def precision_score(y_testing, y_preds):
    tp, fp, tn, fn = _confusion_matrix(y_testing, y_preds)
    precision = tp / (tp + fp)
    return precision


def recall_score(y_testing, y_preds):
    tp, fp, tn, fn = _confusion_matrix(y_testing, y_preds)
    tp_fn = tp + fn
    if tp_fn == 0:
        return 0.0
    else:
        recall = tp / tp_fn
        return recall


def f1_score(y_testing, y_preds):
    precision = precision_score(y_testing, y_preds)
    recall = recall_score(y_testing, y_preds)
    precision_recall = precision + recall
    if precision_recall == 0:
        return 0.0
    else:
        f1 = (2 * precision * recall) / precision_recall
        return f1


def confusion_matrix(y_testing, y_preds):
    tp, fp, tn, fn = _confusion_matrix(y_testing, y_preds)
    con_mat = []
    positives = [tp, fp]
    negatives = [fn, tn]
    con_mat.append(positives)
    con_mat.append(negatives)
    return con_mat

# 7. Main function.

In [ ]:
if __name__ == "__main__":
   zcr, energy, mfcc, spectrals, audios, status = audio_features_status()
   logistic_reg = "logistic_reg_model.pkl"
   knn = "knn_model.pkl"
   svm = "svm_model.pkl"
   decision_tree= "tree_model.pkl"
   random_forest = "random_model.pkl"
   one_dcnn_model = load_model("1dcnn.h5")

   with open(logistic_reg, 'rb') as file:
        logistic_reg_model = pickle.load(file)

   with open(knn, 'rb') as file:
        knn_model = pickle.load(file)

   with open(svm, 'rb') as file:
        svm_model = pickle.load(file)

   with open(decision_tree, 'rb') as file:
        tree_model = pickle.load(file)

   with open(random_forest, 'rb') as file:
        random_model = pickle.load(file)

   logistic_reg_model_preds = logistic_reg_model.predict(zcr)
   knn_model_preds = knn_model.predict(energy)
   svm_model_preds = svm_model.estimate(mfcc)
   tree_model_preds = tree_model.preict(spectrals)
   random_model_preds = random_model.predict(spectrals)

   one_dcnn_model_preds = []
   threshold = 0.5
   for audio in audios:
        pred = one_dcnn_model.predict(audio)
        if pred >= threshold:
            one_dcnn_model_preds.append(1)
        else:
             one_dcnn_model_preds.append(0)

   ensemble_preds = []
   for i in range(len(logistic_reg_model_preds)):
        temp_preds = []
        temp_preds.append(logistic_reg_model_preds[i])
        temp_preds.append(knn_model_preds[i])
        temp_preds.append(svm_model_preds[i])
        temp_preds.append(tree_model_preds[i])
        temp_preds.append(random_model_preds[i])
        temp_preds.append(one_dcnn_model_preds[i])
        ensemble_preds.append(voting(temp_preds))

   ensemble_accuracy = accuracy_score(status, ensemble_preds)
   ensemble_precision = precision_score(status, ensemble_preds)
   ensemble_recall = recall_score(status, ensemble)
   ensemble_f1 = f1_score(status, ensemble_preds)
   ensemble_confusion_mat = confusion_matrix(status, ensemble_preds)

   print("Ensemble accuracy on the testing data:", ensemble_accuracy)
   print("Ensemble precision on the testing data:", ensemble_precision)
   print("Ensemble recall on the testing data:", ensemble_recall)
   print("Ensemble F1 score on the testing fata:", ensemble_f1)
   print("Ensemble confusion matrix on the testing data:", ensemble_confusion_mat)

Exception: ignored